# IEEE Fraud Detection Kaggle Competition
## Adverserial Validation

The objective of this notebook is to create a hold out set using LightGBM to identify rows in train most similar to test

In [1]:
%%bash

ls -l

cd data 

ls -l

total 5530408
-rw-r--r--  1 cchadha2  749149485     1976809 Jul 25 23:44 EDA.ipynb
-rw-r--r--  1 cchadha2  749149485     1661687 Jul 29 22:42 IDs-EDA.ipynb
-rw-r--r--  1 cchadha2  749149485         162 Jul 16 17:57 README.md
-rw-r--r--@ 1 cchadha2  749149485  1401840357 Aug  2 01:59 adverserial_validation.csv
-rw-r--r--  1 cchadha2  749149485      220936 Aug  2 08:03 adverserial_validation.ipynb
-rw-r--r--  1 cchadha2  749149485  1401700566 Aug  2 02:09 adverserial_validation_no_TransactionDT.csv
-rw-r--r--@ 1 cchadha2  749149485        3892 Aug  2 01:59 av_lgbm_importance.csv
-rw-r--r--  1 cchadha2  749149485        3876 Aug  2 02:09 av_lgbm_importance_no_TransactionDT.csv
drwxr-xr-x  8 cchadha2  749149485         256 Jul 17 10:14 data
-rw-r--r--@ 1 cchadha2  749149485       12588 Jul 25 22:31 id_32_nans.png
-rw-r--r--  1 cchadha2  749149485      867988 Jul 30 18:23 log_reg.ipynb
total 220088
-rw-r--r--  1 cchadha2  749149485   1193102 Jul 16 17:16 sample_submission.csv.zip
-rw-r--r--

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [5]:
pd.set_option('display.max_columns', 500)
num_folds = 5
SEED = 5000

In [7]:
train_ids = pd.read_csv('../data/train_identity.csv.zip')
train_trans = pd.read_csv('../data/train_transaction.csv.zip')

test_ids = pd.read_csv('../data/test_identity.csv.zip')
test_trans = pd.read_csv('../data/test_transaction.csv.zip')

In [8]:
train = pd.merge(train_trans, train_ids, on='TransactionID', how='outer')

In [9]:
test = pd.merge(test_trans, test_ids, on='TransactionID', how='outer')

In [10]:
train.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

In [11]:
print(train_trans.shape)

print(train_ids.shape)

print(train.shape)

(590540, 394)
(144233, 41)
(590540, 434)


In [12]:
print(test_trans.shape)

print(test_ids.shape)

print(test.shape)

(506691, 393)
(141907, 41)
(506691, 433)


In [13]:
# Lightgbm parameters
params = {
    'learning_rate': 0.01,
    'max_depth': 5,
    'metric':'auc',
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1
}
early_rounds = 50
num_iterations = 200

In [14]:
train.DeviceInfo.dtypes

dtype('O')

In [15]:
train['isFraud'] = 0
test['isFraud'] = 1
all_data = pd.concat([train, test])

/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
categorical_features=['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
                      'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1',
                      'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
                      'DeviceType', 'DeviceInfo', 'id_12', 'id_13', 'id_14', 'id_15',
                      'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
                      'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
                      'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
                      'id_37', 'id_38']
for feature in categorical_features:
    all_data[feature] = all_data[feature].astype('category')

In [17]:
all_data.shape

(1097231, 434)

In [18]:
all_data.DeviceInfo.dtypes

CategoricalDtype(categories=['0PAJ5', '0PJA2', '0PM92', '1016S', '2014819', '2PQ93',
                  '2PS64', '2PS64 Build/NRD90M', '2PYB2', '2PZC5',
                  ...
                  'verykool_s5516', 'verykools4009', 'verykools5004',
                  'verykools5034', 'verykools5035', 'verykools5036',
                  'verykools5702', 'vivo', 'weimei_we', 'xs-Z47b7VqTMxs'],
                 ordered=False)

In [16]:
# Split train into features and target
y = all_data['isFraud']
X = all_data.drop('isFraud', axis=1)
folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)

In [17]:
all_data.DeviceInfo.dtypes

CategoricalDtype(categories=['0PAJ5', '0PJA2', '0PM92', '1016S', '2014819', '2PQ93',
                  '2PS64', '2PS64 Build/NRD90M', '2PYB2', '2PZC5',
                  ...
                  'verykool_s5516', 'verykools4009', 'verykools5004',
                  'verykools5034', 'verykools5035', 'verykools5036',
                  'verykools5702', 'vivo', 'weimei_we', 'xs-Z47b7VqTMxs'],
                 ordered=False)

In [18]:
# Create arrays for oof predictions and sub predictions
oof_preds = np.zeros(len(X))
feature_importance_df = pd.DataFrame()

In [19]:
# Split data into folds and train
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):

    trn_data = lgb.Dataset(X.iloc[trn_], label=y.iloc[trn_])
    val_data = lgb.Dataset(X.iloc[val_], label=y.iloc[val_])


    clf = lgb.train(params, trn_data, num_iterations, valid_sets=[trn_data, val_data],
                    verbose_eval=50,  early_stopping_rounds = early_rounds)


    oof_preds[val_] = clf.predict(X.iloc[val_], num_iteration=clf.best_iteration)


    # Concatenate fold importances into feature importance dataframe
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X.columns.tolist()
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print('no {}-fold AUC: {}'.format(fold_ + 1, roc_auc_score(y.iloc[val_].values, oof_preds[val_])))

/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.999996	valid_1's auc: 0.999995
[100]	training's auc: 0.999996	valid_1's auc: 0.999996
[150]	training's auc: 0.999997	valid_1's auc: 0.999996
[200]	training's auc: 0.999997	valid_1's auc: 0.999996
Did not meet early stopping. Best iteration is:
[197]	training's auc: 0.999997	valid_1's auc: 0.999996
no 1-fold AUC: 0.9999959856534115


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.999996	valid_1's auc: 0.999996
[100]	training's auc: 0.999996	valid_1's auc: 0.999996
[150]	training's auc: 0.999997	valid_1's auc: 0.999996
[200]	training's auc: 0.999997	valid_1's auc: 0.999996
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.999997	valid_1's auc: 0.999996
no 2-fold AUC: 0.9999960160261313


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.999996	valid_1's auc: 0.999997
Early stopping, best iteration is:
[37]	training's auc: 0.999996	valid_1's auc: 0.999997
no 3-fold AUC: 0.9999967018493091


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.999996	valid_1's auc: 0.999996
[100]	training's auc: 0.999996	valid_1's auc: 0.999996
[150]	training's auc: 0.999997	valid_1's auc: 0.999996
[200]	training's auc: 0.999997	valid_1's auc: 0.999997
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.999997	valid_1's auc: 0.999997
no 4-fold AUC: 0.9999966444084325


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.999996	valid_1's auc: 0.999995
[100]	training's auc: 0.999996	valid_1's auc: 0.999996
[150]	training's auc: 0.999997	valid_1's auc: 0.999996
[200]	training's auc: 0.999997	valid_1's auc: 0.999996
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.999997	valid_1's auc: 0.999996
no 5-fold AUC: 0.9999958672229298


In [20]:
print('OVERALL AUC: {:.5f}'.format(roc_auc_score(y, oof_preds)))

all_data['preds'] = oof_preds
all_data.to_csv('adverserial_validation.csv', index=False)

OVERALL AUC: 0.99976


In [21]:
feature_importance_df = feature_importance_df.groupby('feature', as_index=False).mean().drop('fold', axis=1).sort_values(
                     'importance', ascending=False)

In [22]:
feature_importance_df.head()

,feature,importance
0,C1,252.8
44,TransactionDT,214.2
20,D15,137.6
28,D9,127.6
45,TransactionID,119.4


In [23]:
# Save average feature importances
feature_importance_df.to_csv('av_lgbm_importance.csv', index=False)

In [5]:
train.to_csv('merged_train.csv', index=False)
test.to_csv('merged_test.csv', index=False)

# Removing TransactionDT, TransactionID, and predictions

In [20]:
# Split train into features and target
y = all_data['isFraud']
# X = all_data.drop(['preds'], axis=1)
X = all_data.drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)

In [21]:
all_data.DeviceInfo.dtypes

CategoricalDtype(categories=['0PAJ5', '0PJA2', '0PM92', '1016S', '2014819', '2PQ93',
                  '2PS64', '2PS64 Build/NRD90M', '2PYB2', '2PZC5',
                  ...
                  'verykool_s5516', 'verykools4009', 'verykools5004',
                  'verykools5034', 'verykools5035', 'verykools5036',
                  'verykools5702', 'vivo', 'weimei_we', 'xs-Z47b7VqTMxs'],
                 ordered=False)

In [22]:
# Create arrays for oof predictions and sub predictions
oof_preds = np.zeros(len(X))
feature_importance_df = pd.DataFrame()

In [ ]:
# Split data into folds and train
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):

    trn_data = lgb.Dataset(X.iloc[trn_], label=y.iloc[trn_], categorical_feature=categorical_features)
    val_data = lgb.Dataset(X.iloc[val_], label=y.iloc[val_])


    clf = lgb.train(params, trn_data, num_iterations, valid_sets=[trn_data, val_data], verbose_eval=1000,
                    early_stopping_rounds=early_rounds)


    oof_preds[val_] = clf.predict(X.iloc[val_], num_iteration=clf.best_iteration)


    # Concatenate fold importances into feature importance dataframe
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X.columns.tolist()
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print('no {}-fold AUC: {}'.format(fold_ + 1, roc_auc_score(y.iloc[val_].values, oof_preds[val_])))

/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DeviceInfo', 'DeviceType', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical

Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.874302	valid_1's auc: 0.874175
no 1-fold AUC: 0.8741753493168642


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DeviceInfo', 'DeviceType', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical

Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.874672	valid_1's auc: 0.87345
no 2-fold AUC: 0.873450291814792


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DeviceInfo', 'DeviceType', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical

Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.873828	valid_1's auc: 0.874208
no 3-fold AUC: 0.8742077987418042


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DeviceInfo', 'DeviceType', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical

Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[200]	training's auc: 0.87464	valid_1's auc: 0.873545
no 4-fold AUC: 0.8735452199023337


/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['DeviceInfo', 'DeviceType', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'P_emaildomain', 'ProductCD', 'R_emaildomain', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/Users/cchadha2/anaconda3/envs/eda/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical

Training until validation scores don't improve for 50 rounds.


In [ ]:
print('OVERALL AUC: {:.5f}'.format(roc_auc_score(y, oof_preds)))

all_data['preds'] = oof_preds
# all_data.to_csv('adverserial_validation_no_TransactionDT.csv', index=False)

In [37]:
feature_importance_df = feature_importance_df.groupby('feature', as_index=False).mean().drop('fold', axis=1).sort_values(
                     'importance', ascending=False)

In [38]:
feature_importance_df.head()

,feature,importance
20,D15,553.6
423,id_31,532.0
405,id_13,220.4
393,id_01,177.2
18,D13,177.0


In [39]:
# Save average feature importances
feature_importance_df.to_csv('av_lgbm_importance_no_TransactionDT.csv', index=False)

In [ ]:
all_data[(all_data['isFraud'] == 0) & (all_data['preds'] > 0.6)]

In [72]:
all_data[(all_data['isFraud'] == 0) & (all_data['preds'] > 0.6)].shape

(7171, 435)

In [ ]:
# val_indices = all_data[(all_data['isFraud'] == 0) & (all_data['preds'] > 0.6)].index
val_indices = all_data[(all_data['isFraud'] == 0) & (all_data['preds'] > 0.5)].index

In [ ]:
val_indices = np.asarray(val_indices)

In [ ]:
np.save('../output/val_indices_>0.5', val_indices)